# Кредитный скоринг
### тест на обзвонах апреля 2017


In [245]:
##### Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import hashlib
from collections import Counter

hive_config_query = '''
set hive.vectorized.execution.enabled=true;
set hive.vectorized.execution.reduce.enabled = true;
set mapreduce.map.memory.mb=4096;
set mapreduce.map.child.java.opts=-Xmx4g;
set mapreduce.task.io.sort.mb=1024;
set mapreduce.reduce.child.java.opts=-Xmx4g;
set mapreduce.reduce.memory.mb=7000;
set mapreduce.reduce.shuffle.input.buffer.percent=0.5;
set mapreduce.input.fileinputformat.split.minsize=536870912;
set mapreduce.input.fileinputformat.split.maxsize=1073741824;
set hive.optimize.ppd=true;
set hive.merge.smallfiles.avgsize=536870912;
set hive.merge.mapredfiles=true;
set hive.merge.mapfiles=true;
set hive.hadoop.supports.splittable.combineinputformat=true;
set hive.exec.reducers.bytes.per.reducer=536870912;
set hive.exec.parallel=true;
set hive.exec.max.created.files=10000000;
set hive.exec.compress.output=true;
set hive.exec.dynamic.partition.mode=nonstrict;
set hive.exec.max.dynamic.partitions=1000000;
set hive.exec.max.dynamic.partitions.pernode=100000;
set io.seqfile.compression.type=BLOCK;
set mapreduce.map.failures.maxpercent=5;
'''

sc.stop()
conf = (SparkConf()
        .set("spark.executor.instances", 2)
        .set("spark.driver.maxResultSize", "4g")
        .set('spark.driver.memory','4g')
        .set("spark.executor.memory", '2g')
        .set("spark.yarn.executor.memoryOverhead", 1048)
       )
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

for q in hive_config_query.split(';'):
    try:
        hc.sql(q)
    except:
        pass

In [204]:
# different datasets (varying history depth)
options = ((1,180,'2016-12-01'),(2,60,'2000-01-01'),(3,30,'2000-01-01'),(4,10,'2000-01-01'),(5,2,'2000-01-01'))
#print('\n\n'.join(queries))


In [205]:

def metrics(y_true,y_score,lift = None, return_str = False):
    import sklearn
    import collections
    
    if True:
        
        res = collections.OrderedDict()
        samp_size = len(y_true)
        res['Sample size'] = samp_size
        res['Posit share'] = sum(y_true) * 1./ samp_size
        res['Sample size'] = len(y_true)
        res['AUC ROC'] = sklearn.metrics.roc_auc_score(y_true = y_true, y_score = y_score)
        res['AUC PR'] = sklearn.metrics.average_precision_score( y_true,  y_score)
        res['Log loss'] = sklearn.metrics.log_loss(y_true = y_true, y_pred = y_score)
        if lift:
            predictions_and_labels = sorted(zip(y_score,y_true), key = lambda e:-e[0])
            for l in lift:
                res['Lift ' + str(l)] = sum([e[1] for e in predictions_and_labels[:int(l * samp_size)]]) * 1. / int(l * samp_size) / res['Posit share']                
        if return_str:
            res = '\n'.join(['{:<12}: {:.5f}'.format(k,v) for (k,v) in res.items()]) + '.'
        return res

In [206]:
cols = [u'phone_mobile', u'call_ymd', u'approve', u'cnt', u'visits_cnt',
       u'hits', u'avg_duration', u'avg_time_std', u'avg_ymd_range',
       u'avg_ymd_cnt', u'avg_hour', u'avg_hour_q10', u'avg_hour_q90',
       u'max_score1', u'avg_score1', u'q95_score1', u'q90_score1',
       u'q70_score1', u'q50_score1', u'q30_score1', u'max_score2',
       u'avg_score2', u'q95_score2', u'q90_score2', u'q70_score2',
       u'q50_score2', u'q30_score2', u'max_score3', u'avg_score3',
       u'q95_score3', u'q90_score3', u'q70_score3', u'q50_score3',
       u'q30_score3',  u'max_score4', u'avg_score4',
       u'q95_score4', u'q90_score4', u'q70_score4', u'q50_score4',
       u'q30_score4', u'emailru', u'mobile_share', u'vk_share',
       u'social_share', u'work_hours_hits_share', u'hour_std',
       u'good_urlfr_share_score1', u'good_urlfr_share_score2',
       u'good_urlfr_share_score3', u'good_urlfr_timestd_score1',
       u'max_urlfr_time_std_1', u'pop_country_share',
       u'pop_city_share', u'density', u'area_sq_km', u'federal_district',
       u'avg_salary_2015_rub', u'utc_time_zone_val', u'yaca_str',
        u'min_score1', u'min_score2', u'min_score3', u'min_score4']
feat_cols = cols[2:]

In [210]:
import cPickle
from sklearn.feature_extraction import DictVectorizer
import os
import sklearn, sklearn.cross_validation

v1 = cPickle.load(open('data/ccall_scoring_dict_vectorizer','r'))
type(v1)
federal_districts = [u'ЦФО',u'СЗФО',u'ЮФО',u'СКФО',u'ПФО',u'УФО',u'СФО',u'ДВФО'] # todo

def encode(v, classes, default_value = -1):
    '''Encode text value v which values are from classes list. Returns v index and -1 if it wasn't found in the list.'''
    try:
        return classes.index(v)
    except ValueError:
        return default_value
from pylightgbm.models import GBMClassifier

exec_path = "/opt/share/LightGBM-master/lightgbm"
os.environ["LIGHTGBM_EXEC"] = exec_path

In [211]:
#old script
def cv(clf, X, y, folds = 5, metrics = 'roc_auc,pr_auc'):
    '''calc cross-validation metrics for clf classfier (actually LightGBM classifier) on X,y data '''
    assert X.shape[0] == len(y), 'X and y lengths doesnt match'
    idx = range(X.shape[0])
    np.random.shuffle(idx)
    fold_idx = []
    for i in range(folds):
        fold_idx.append(idx[i*len(idx)/folds:(i+1)*len(idx)/folds])
    res = {k:[] for k in metrics.split(',')}
    
    for i in range(folds):
        train_idx = reduce(lambda x,y: x+y,(fold_idx[:i] + fold_idx[(i+1):]))
        valid_idx = fold_idx[i]
        
        clf.fit(X[train_idx],y[train_idx])
        valid_pred = clf.predict_proba(X[valid_idx])[:,1]
        
        if('roc_auc' in metrics):
            res['roc_auc'].append(
                sklearn.metrics.roc_auc_score(
                  y_true = y[valid_idx],
                  y_score = valid_pred
                )
            )
        if('pr_auc' in metrics):
            res['pr_auc'].append(
                sklearn.metrics.average_precision_score(
                  y_true = y[valid_idx],
                  y_score = valid_pred
                )
            )
    return res

## Выборка по 2 месяцам

In [26]:
feat_select_res = {}
df_train,df_test,df_train_yaca_dense,df_test_yaca_dense = {}, {},{},{}

for ind,depth,_ in options[1:2]: #((3,30,'2000-01-01'),):
    df_train[ind] = (hc.sql('select * from user_kposminin.ccall_scoring_opt_new#ind_scoring_add'.replace('#ind',str(ind)))
        .filter('call_ymd <= "2016-11-15"')
        .map(lambda r: 
                list(r[:15]) + (r[15] if r[15] else [None] * 5) + list(r[16:18]) + (r[18] if r[18] else  [None] * 5)  + list(r[19:21]) + 
               (r[21] if r[21]  else [None] * 5)  + list(r[22:24]) + (r[24] if r[24]  else [None] * 5) + list(r[25:40]) + 
                [encode(r[40], federal_districts)] + list(r[41:])
            )
        .toDF()
        .toPandas()
         )
                
    df_train[ind].columns = cols
    #feat_cols = [c for c in df_train.columns if not c in (u'phone_mobile', u'call_ymd', u'approve', u'yaca_str')]
    
    #Handle yaca columns
    df_train_yaca_dense[ind] = v1.transform(df_train[ind]['yaca_str'].map(lambda s: { kv.split(':')[0]:float(kv.split(':')[1])  for kv in s.split(' ')} if s else {}))
    #yaca_cols = ['yaca_{}'.format(i) for i in range(df_train_yaca_dense.shape[1])]
    for i in range(df_train_yaca_dense[ind].shape[1]):
        df_train[ind].loc[:,'yaca_{}'.format(i)] = df_train_yaca_dense[ind][:,i]
    #feat_cols_w_yaca = feat_cols + yaca_cols
    
    #Test data formation
    df_test[ind] = (hc.sql('select * from user_kposminin.ccall_scoring_opt_new#ind_scoring_add'.replace('#ind',str(ind)))
        .filter(' call_ymd > "2016-11-15"')
        .map(lambda r: 
                list(r[:15]) + (r[15] if r[15] else [None] * 5) + list(r[16:18]) + (r[18] if r[18] else  [None] * 5)  + list(r[19:21]) + 
               (r[21] if r[21]  else [None] * 5)  + list(r[22:24]) + (r[24] if r[24]  else [None] * 5) + list(r[25:40]) + 
                [encode(r[40], federal_districts)] + list(r[41:])
            )
        .toDF()
        .toPandas()
         )
    
    df_test[ind].columns = cols
    # Handling categrial text columns
    #for i in categorial_cols:
    #    df_test.iloc[:,i] = enc[i].transform(df_test.iloc[:,i])
        
    #Handle yaca columns
    df_test_yaca_dense[ind] = v1.transform(df_test[ind]['yaca_str'].map(lambda s: { kv.split(':')[0]:float(kv.split(':')[1])  for kv in s.split(' ')} if s else {}))
    #yaca_cols = ['yaca_{}'.format(i) for i in range(df_test_yaca_dense.shape[1])]
    for i in range(df_test_yaca_dense[ind].shape[1]):
        df_test[ind].loc[:,'yaca_{}'.format(i)] = df_test_yaca_dense[ind][:,i]
    


In [175]:
#Selected option
ind = 2
feats = [u'cnt',
 u'visits_cnt',
 u'hits',
 u'max_score1',
 u'avg_score1',
 u'max_score2',
 u'avg_score2',
 u'max_score3',
 u'avg_score3',
 u'max_score4',
 u'avg_score4',
 u'emailru',
 u'mobile_share',
 u'vk_share',
 u'social_share',
 u'work_hours_hits_share',
 u'hour_std',
 u'good_urlfr_share_score1',
 u'good_urlfr_share_score2',
 u'good_urlfr_share_score3',
 u'pop_country_share',
 u'pop_city_share',
 u'density',
 u'area_sq_km',
 u'federal_district',
 u'avg_salary_2015_rub',
 u'utc_time_zone_val',
 u'min_score1',
 u'min_score2',
 u'min_score3',
 u'min_score4']

In [126]:
clf = GBMClassifier(
        exec_path=exec_path,
        min_data_in_leaf=50,
       # is_unbalance = True,
        num_iterations = 120,
        bagging_fraction = 0.8,
        bagging_freq = 10,
        num_leaves = 127,
        learning_rate = 0.05,
        metric = 'auc',
    )

clf.fit(df_train[ind][feats], df_train[ind][label], test_data = [(df_test[ind][feats], df_test[ind][label])])

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Warning] Ignoring feature Column_13, only has one value
[LightGBM] [Info] Finished loading data in 3.446603 seconds
[LightGBM] [Info] Number of postive: 118966, number of negative: 212160
[LightGBM] [Info] Number of data: 331126, number of features: 30
[LightGBM] [Info] Finished initializing training
[LightGBM] [Info] Started training...
[LightGBM] [Info] Iteration:1, valid_1 auc : 0.652865
[LightGBM] [Info] 0.178056 seconds elapsed, finished iteration 1
[LightGBM] [Info] Iteration:2, valid_1 auc : 0.656721
[LightGBM] [Info] 0.356666 seconds elapsed, finished iteration 2
[LightGBM] [Info] Iteration:3, valid_1 auc : 0.657199
[LightGBM] [Info] 0.559029 seconds elapsed, finished iteration 3
[LightGBM] [Info] Iteration:4, valid_1 auc : 0.658428
[LightGBM] [Info] 0.849206 seconds elapsed, finished iteration 4
[LightGBM] [Info] Iteration:5, valid_1 auc : 0.658534
[LightGBM] [Info] 1.023089 seconds elapsed, finished iteration 5
[LightG

In [61]:
all_feats = [c for c in df_train[ind].columns[3:].tolist() if c != u'yaca_str']

In [63]:
clf = GBMClassifier(
        exec_path=exec_path,
        min_data_in_leaf=50,
       # is_unbalance = True,
        num_iterations = 250,
        bagging_fraction = 0.8,
        bagging_freq = 10,
        num_leaves = 127,
        learning_rate = 0.05,
        metric = 'auc',
    )

clf.fit(df_train[ind][all_feats], df_train[ind][label], test_data = [(df_test[ind][all_feats], df_test[ind][label])])

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Warning] Ignoring feature Column_40, only has one value
[LightGBM] [Warning] Ignoring feature Column_126, only has one value
[LightGBM] [Warning] Ignoring feature Column_136, only has one value
[LightGBM] [Warning] Ignoring feature Column_181, only has one value
[LightGBM] [Warning] Ignoring feature Column_186, only has one value
[LightGBM] [Warning] Ignoring feature Column_214, only has one value
[LightGBM] [Warning] Ignoring feature Column_304, only has one value
[LightGBM] [Warning] Ignoring feature Column_355, only has one value
[LightGBM] [Warning] Ignoring feature Column_403, only has one value
[LightGBM] [Warning] Ignoring feature Column_466, only has one value
[LightGBM] [Warning] Ignoring feature Column_467, only has one value
[LightGBM] [Warning] Ignoring feature Column_469, only has one value
[LightGBM] [Info] Finished loading data in 33.654988 seconds
[LightGBM] [Info] Number of postive: 118966, number of negative: 2

### Итак, использование 31 аддитивного параметра имеет AUC ROC  0.67025 на тесте, в то время как все параметры 0.678291. Снижение есть, но с ним можно жить.

### Тренируем классификатор

In [68]:
all_data = pd.concat([df_train[ind][[u'phone_mobile',u'call_ymd',u'approve'] + feats],df_test[ind][[u'phone_mobile',u'call_ymd',u'approve'] + feats]])

In [156]:
clf = GBMClassifier(
        exec_path=exec_path,
        min_data_in_leaf=50,
       # is_unbalance = True,
        num_iterations = 120,
        bagging_fraction = 0.8,
        bagging_freq = 10,
        num_leaves = 127,
        learning_rate = 0.05,
        metric = 'auc',
    )

clf.fit(all_data[feats], all_data[label])

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Warning] Ignoring feature Column_13, only has one value
[LightGBM] [Info] Finished loading data in 3.610838 seconds
[LightGBM] [Info] Number of postive: 136669, number of negative: 239299
[LightGBM] [Info] Number of data: 375968, number of features: 30
[LightGBM] [Info] Finished initializing training
[LightGBM] [Info] Started training...
[LightGBM] [Info] 0.195577 seconds elapsed, finished iteration 1
[LightGBM] [Info] 0.392972 seconds elapsed, finished iteration 2
[LightGBM] [Info] 0.611375 seconds elapsed, finished iteration 3
[LightGBM] [Info] 0.823941 seconds elapsed, finished iteration 4
[LightGBM] [Info] 1.042539 seconds elapsed, finished iteration 5
[LightGBM] [Info] 1.258831 seconds elapsed, finished iteration 6
[LightGBM] [Info] 1.465479 seconds elapsed, finished iteration 7
[LightGBM] [Info] 1.659068 seconds elapsed, finished iteration 8
[LightGBM] [Info] 1.871416 seconds elapsed, finished iteration 9
[LightGBM] [Info]

In [73]:
#cPickle.dump(clf,open('data/cred_scor_LGBM_classifier.pck','w'))

In [212]:
clf_orig = cPickle.load(open('data/cred_scor_LGBM_classifier.pck','r'))

In [154]:
df_test = (hc.sql('select * from user_kposminin.id_cred_scor_test_201703')        
        .toPandas()
         )

In [87]:
print('liru cookie found share {:.2%}, features found share {:.2%}'.format(
          df_test_apr2017.id_cnt.map(lambda v:v>0).mean(),
        df_test_apr2017.acc_id_cnt.map(lambda v:v>0).mean())
     )

liru cookie found share 37.63%, features found share 21.06%


In [155]:
test_data = df_test.query('acc_id_cnt > 0')
test_data.loc[:,'federal_district'] = test_data.federal_district.map(lambda v:encode(v, federal_districts))

In [157]:
test_data.shape

(90278, 42)

In [158]:
test_data['pred'] = clf.predict_proba(test_data[feats])[:,1]

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 120 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction


In [159]:
print(metrics(y_true = test_data.approve, y_score = test_data.pred, lift = None, return_str = True))

Sample size : 90278.00000
Posit share : 0.41057
AUC ROC     : 0.55992
AUC PR      : 0.45522
Log loss    : 0.70412.


 ## провал

In [130]:
#test_data.query('utm_campaign > "cold" and utm_campaign < "cole"')
#test_data.groupby('utm_campaign').phone_mobile.count().sort(inplace=False)

In [194]:
df_test_29apr2017 = (hc.sql('select * from user_kposminin.id_cred_scor_test_20170429')        
        .toPandas()
         )

In [138]:
df_test_29apr2017.loc[:,'federal_district'] = df_test_29apr2017.federal_district.map(lambda v:encode(v, federal_districts))
df_test_29apr2017['pred'] = clf.predict_proba(df_test_29apr2017[feats])[:,1]

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 120 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction


In [139]:
print(metrics(y_true = df_test_29apr2017.approve, y_score = df_test_29apr2017.pred, lift = None, return_str = True))

Sample size : 1928.00000
Posit share : 0.48081
AUC ROC     : 0.55939
AUC PR      : 0.52516
Log loss    : 0.70219.


In [142]:
df_test_29apr2017.approve.count()

1928

In [160]:
data = df_test.query('acc_id_cnt > 0')
data.loc[:,'federal_district'] = data.federal_district.map(lambda v:encode(v, federal_districts))
train_data = data.query('call_ymd <= "2017-04-23"')
test_data = data.query('call_ymd > "2017-04-23"')
train_data.shape,test_data.shape

((70259, 42), (20019, 42))

In [161]:
clf = GBMClassifier(
        exec_path=exec_path,
        min_data_in_leaf=50,
       # is_unbalance = True,
        num_iterations = 120,
        bagging_fraction = 0.8,
        bagging_freq = 10,
        num_leaves = 127,
        learning_rate = 0.05,
        metric = 'auc',
    )

clf.fit(train_data[feats], train_data[label])

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Warning] Ignoring feature Column_13, only has one value
[LightGBM] [Info] Finished loading data in 1.124925 seconds
[LightGBM] [Info] Number of postive: 28643, number of negative: 41616
[LightGBM] [Info] Number of data: 70259, number of features: 30
[LightGBM] [Info] Finished initializing training
[LightGBM] [Info] Started training...
[LightGBM] [Info] 0.049639 seconds elapsed, finished iteration 1
[LightGBM] [Info] 0.100280 seconds elapsed, finished iteration 2
[LightGBM] [Info] 0.148198 seconds elapsed, finished iteration 3
[LightGBM] [Info] 0.197885 seconds elapsed, finished iteration 4
[LightGBM] [Info] 0.245239 seconds elapsed, finished iteration 5
[LightGBM] [Info] 0.296450 seconds elapsed, finished iteration 6
[LightGBM] [Info] 0.346647 seconds elapsed, finished iteration 7
[LightGBM] [Info] 0.393662 seconds elapsed, finished iteration 8
[LightGBM] [Info] 0.441604 seconds elapsed, finished iteration 9
[LightGBM] [Info] 0.

In [162]:
test_data['pred'] = clf.predict_proba(test_data[feats])[:,1]

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 120 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction


/opt/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [163]:
print(metrics(y_true = test_data.approve, y_score = test_data.pred, lift = None, return_str = True))

Sample size : 20019.00000
Posit share : 0.42070
AUC ROC     : 0.63925
AUC PR      : 0.54050
Log loss    : 0.64931.


#### Итак, модель, посчитанная новым способом, набирает качество с увеличением объема выборки. Может, доберется до целевого показателя 0.68. Но это надо считать с 2015 года, что долго (можно взять только полные заявки).

In [166]:
df_20170429_aza.columns

Index([u'phone_mobile', u'call_ymd', u'cnt', u'visits_cnt', u'hits',
       u'avg_duration', u'avg_time_std', u'avg_ymd_range', u'avg_ymd_cnt',
       u'avg_hour', u'avg_hour_q10', u'avg_hour_q90', u'max_score1',
       u'avg_score1', u'q_arr_score1', u'max_score2', u'avg_score2',
       u'q_arr_score2', u'max_score3', u'avg_score3', u'q_arr_score3',
       u'emailru', u'mobile_share', u'vk_share', u'social_share',
       u'work_hours_hits_share', u'hour_std', u'good_urlfr_share_score1',
       u'good_urlfr_share_score2', u'good_urlfr_share_score3',
       u'good_urlfr_timestd_score1', u'max_urlfr_time_std_1', u'yaca_str',
       u'mob_provider', u'ind', u'pop_country_share', u'pop_city_share',
       u'density', u'area_sq_km', u'federal_district', u'avg_salary_2015_rub',
       u'utc_time_zone_val'],
      dtype='object')

In [213]:
df_20170429_aza2 = (hc.sql('select * from user_kposminin.cred_scor_calc_debug_20170429_scoring_new_add')        
        .toPandas()
         )
df_20170429_aza2.loc[:,'federal_district'] = df_20170429_aza.federal_district.map(lambda v:encode(v, federal_districts))
df_20170429_aza2['pred'] = clf_orig.predict_proba(df_20170429_aza2[feats])[:,1]
#print(metrics(y_true = df_20170429_aza2.approve, y_score = df_20170429_aza2.pred, lift = None, return_str = True))

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 120 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction


/opt/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:13: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal


,phone_mobile,id_cnt,acc_id_cnt,call_ymd,approve,utilization,in_work,utm_campaign,considered,cnt,...,good_urlfr_share_score3,mob_provider,ind,pop_country_share,pop_city_share,density,area_sq_km,federal_district,avg_salary_2015_rub,utc_time_zone_val
0,89000119089,1,1,2017-04-29,0,0,0,cold_liru_test,1,1056,...,0.004735,Tele2,37,0.0089,0.7543,15.495588,84201,0,23579,3
1,89000127667,1,1,2017-04-29,1,0,0,cold_liru,1,1944,...,0.002058,Tele2,37,0.0089,0.7543,15.495588,84201,0,23579,3
2,89000147282,1,1,2017-04-29,0,0,0,None,1,889,...,0.000000,Tele2,37,0.0089,0.7543,15.495588,84201,0,23579,3
3,89000165702,2,2,2017-04-29,1,0,0,cold_psp_rj,1,828,...,0.000000,Tele2,37,0.0089,0.7543,15.495588,84201,0,23579,3
4,89000220343,2,2,2017-04-29,1,0,0,cold_mega_void,1,3979,...,0.000754,Tele2,9,0.0239,0.8259,39.543155,88529,5,24464,5
5,89000507981,3,3,2017-04-29,0,0,0,None,1,7659,...,0.002220,Tele2,16,0.0185,0.8577,28.389940,95725,6,21553,7
6,89000511745,1,1,2017-04-29,0,0,0,cold_liru_test,1,891,...,0.025814,Tele2,16,0.0185,0.8577,28.389940,95725,6,21553,7
7,89000616953,1,1,2017-04-29,0,0,0,cold_skip,1,171,...,0.000000,Tele2,9,0.0239,0.8259,39.543155,88529,5,24464,5
8,89000719228,1,1,2017-04-29,0,0,0,cold_skip,1,2231,...,0.000448,Tele2,9,0.0239,0.8259,39.543155,88529,5,24464,5
9,89001046009,1,1,2017-04-29,0,0,0,cold_skip,1,244,...,0.004098,Tele2,16,0.0185,0.8577,28.389940,95725,6,21553,7


### Сравнение старого (aza) расчета и нового (accum) для выборки - полных заявок за 2017-04-29.

In [221]:
#df_aza = df_20170429_aza2
#df_acc = hc.sql("select * from user_kposminin.id_cred_scor_test_20170429 where call_ymd = '2017-04-29' and acc_id_cnt>0").toPandas()
#df_acc.loc[:,'federal_district'] = df_acc.federal_district.map(lambda v:encode(v, federal_districts))
df_acc = df_acc.rename(columns = {'urlfr_cnt':u'cnt',u'emailru_sum': u'emailru'})
df_acc['pred'] = clf_orig.predict_proba(df_acc[feats])[:,1]

df_aza.shape,df_acc.shape

[LightGBM] [Info] Finished loading parameters
[LightGBM] [Info] Finished loading 120 models
[LightGBM] [Info] Finished initializing prediction
[LightGBM] [Info] Finished prediction


((1928, 49), (1928, 43))

In [227]:
acc_row = df_acc.query('phone_mobile == "89266630611"').iloc[0]
aza_row = df_aza.query('phone_mobile == "89266630611"').iloc[0]

In [237]:
pd.DataFrame([acc_row[feats],aza_row[feats]]).T

,1000,1000
cnt,2583.000000,1718.000000
visits_cnt,2583.000000,2583.000000
hits,9680.000000,9680.000000
max_score1,3.044523,3.044523
avg_score1,-0.582262,-0.626780
max_score2,-5.847996,-5.847996
avg_score2,-10.990930,-11.176768
max_score3,-3.380712,-3.380712
avg_score3,-8.728679,-8.607945
max_score4,-1.087623,-0.646627


In [225]:
=print(metrics(y_true = df_aza.approve, y_score = df_aza.pred, lift = None, return_str = True))

Sample size : 1928.00000
Posit share : 0.48081
AUC ROC     : 0.64993
AUC PR      : 0.59896
Log loss    : 0.66059.


In [226]:
print(metrics(y_true = df_acc.approve, y_score = df_acc.pred, lift = None, return_str = True))

Sample size : 1928.00000
Posit share : 0.48081
AUC ROC     : 0.62264
AUC PR      : 0.58081
Log loss    : 0.71297.


In [244]:
pd.DataFrame([df_acc.query('phone_mobile == "89000165702"').iloc[0][feats],df_aza.query('phone_mobile == "89000165702"').iloc[0][feats]]).T

,3,3
cnt,828.000000,410.000000
visits_cnt,828.000000,4968.000000
hits,5400.000000,32400.000000
max_score1,3.044523,3.044523
avg_score1,-0.753522,-0.919542
max_score2,-9.566122,-9.566122
avg_score2,-12.037147,-11.778418
max_score3,-8.393448,-8.393448
avg_score3,-9.551495,-9.395360
max_score4,-1.301841,2.397895
